In [1]:
import os

In [2]:
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\Admin\Desktop\Rohit\MachineLearning\nlp-learning\fake news project


In [11]:
os.chdir('./fake news dataset')
print("Changed Working Directory:", os.getcwd())

Changed Working Directory: c:\Users\Admin\Desktop\Rohit\MachineLearning\nlp-learning\fake news dataset


In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('train.csv')

In [15]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
df.shape

(20800, 5)

In [18]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [20]:
df = df.dropna()

In [21]:
df.shape

(18285, 5)

In [22]:
X = df.drop('label', axis = 1)
y = df['label']

In [25]:
X.shape

(18285, 4)

In [26]:
y.shape

(18285,)

In [27]:
import tensorflow as tf

In [28]:
tf.__version__

'2.16.2'

In [29]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [30]:
voc_size = 5000

In [31]:
messages = X.copy()

In [32]:
messages.reset_index(inplace = True)

In [71]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [35]:
import nltk
import re
from nltk.corpus import stopwords

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [39]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [72]:
corpus[0:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [42]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
# onehot_repr

### Embedding Representation

In [43]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1967 1904  302]
 [   0    0    0 ...  180  268 1765]
 [   0    0    0 ... 4637 4226 1936]
 ...
 [   0    0    0 ... 1224 4044 2427]
 [   0    0    0 ... 2020 3167 1833]
 [   0    0    0 ...  275 3907  445]]


In [44]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1066,
       1556,  673, 1391, 1094, 3947, 3321, 1967, 1904,  302])

### Model Creation

In [51]:
embedding_vector_features=40
model=Sequential()
# model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features))
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [52]:
len(embedded_docs),y.shape

(18285, (18285,))

In [53]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [55]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.7895 - loss: 0.4287 - val_accuracy: 0.9089 - val_loss: 0.2028
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9405 - loss: 0.1462 - val_accuracy: 0.9173 - val_loss: 0.1981
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9636 - loss: 0.0999 - val_accuracy: 0.9155 - val_loss: 0.2151
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9755 - loss: 0.0748 - val_accuracy: 0.9168 - val_loss: 0.2471
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9808 - loss: 0.0561 - val_accuracy: 0.9074 - val_loss: 0.2889
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9871 - loss: 0.0421 - val_accuracy: 0.9087 - val_loss: 0.3180
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9894 - loss: 0.0320 - val_accuracy: 0.9082 - val_loss: 0.3802
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9939 - loss: 0.0232 - val_acc

### Adding Dropout

In [56]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [66]:
y_pred_prob=model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [67]:
y_test, y_pred

(array([1, 0, 0, ..., 0, 1, 1], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [68]:
from sklearn.metrics import confusion_matrix

In [69]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]], dtype=int64)

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917